## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

#get config variable from folder above
os.chdir("..")
from config import g_key
os.chdir("Vacation_Search")

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the Weather_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Flinders,AU,-34.5833,150.8552,65.37,91,73,4.00,broken clouds
1,1,Alofi,NU,-19.0595,-169.9187,80.49,89,91,3.44,overcast clouds
2,2,San Quintin,MX,30.4833,-115.9500,80.20,25,72,8.30,broken clouds
3,3,Narsaq,GL,60.9167,-46.0500,37.62,70,39,5.03,scattered clouds
4,4,Ugoofaaru,MV,5.6667,73.0000,84.54,66,28,5.23,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp_in = float(input("What is the minimum temperature you would like for your trip? "))
max_temp_in = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_raw_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp_in) & \
                                       (city_data_df["Max Temp"] >= min_temp_in)]
preferred_cities_raw_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Kapaa,US,22.0752,-159.3190,78.78,80,75,20.71,broken clouds
13,13,Mar Del Plata,AR,-38.0023,-57.5575,70.09,92,75,7.00,broken clouds
21,21,Rikitea,PF,-23.1203,-134.9692,79.52,73,6,17.78,light rain
22,22,Bengkulu,ID,-3.8004,102.2655,74.53,89,100,3.71,overcast clouds
23,23,Korla,CN,41.7597,86.1469,70.92,22,100,12.82,overcast clouds
24,24,Hambantota,LK,6.1241,81.1185,79.57,79,96,7.16,overcast clouds
26,26,Atuona,PF,-9.8000,-139.0333,79.47,78,90,18.12,overcast clouds
28,28,Opuwo,NaN,-18.0607,13.8400,76.23,37,4,4.25,clear sky
29,29,Labuhan,ID,-6.8844,112.2051,77.09,88,75,9.93,broken clouds
30,30,Jambi,ID,-1.6000,103.6167,73.60,97,96,2.04,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_raw_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_raw_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_raw_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_raw_df["Hotel Name"] = ""
hotel_raw_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
13,Mar Del Plata,AR,70.09,broken clouds,-38.0023,-57.5575,
21,Rikitea,PF,79.52,light rain,-23.1203,-134.9692,
22,Bengkulu,ID,74.53,overcast clouds,-3.8004,102.2655,
23,Korla,CN,70.92,overcast clouds,41.7597,86.1469,
24,Hambantota,LK,79.57,overcast clouds,6.1241,81.1185,
26,Atuona,PF,79.47,overcast clouds,-9.8000,-139.0333,
29,Labuhan,ID,77.09,broken clouds,-6.8844,112.2051,
30,Jambi,ID,73.60,overcast clouds,-1.6000,103.6167,
32,Cidreira,BR,70.02,overcast clouds,-30.1811,-50.2056,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_raw_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_raw_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_raw_df.count())
hotel_df=hotel_raw_df.loc[hotel_raw_df["Hotel Name"] != ""]
print(hotel_df.count())
hotel_df.head()

City                   158
Country                158
Max Temp               158
Current Description    158
Lat                    158
Lng                    158
Hotel Name             158
dtype: int64
City                   150
Country                150
Max Temp               150
Current Description    150
Lat                    150
Lng                    150
Hotel Name             150
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Mar Del Plata,AR,70.09,broken clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
21,Rikitea,PF,79.52,light rain,-23.1203,-134.9692,People ThankYou
22,Bengkulu,ID,74.53,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
23,Korla,CN,70.92,overcast clouds,41.7597,86.1469,Gulisitan Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))